# How to analyze HTO data

In our database, we utilized HTO data generated from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE251912. This documentation has instruction on how we use HTO data to demultiplex the gene expression samples. <br>
This analysis can be conducted using the script `scripts/HTODemux.R` with appropriate modification, as noted in this documentation.

## Some metadata about this dataset is the following:

1. The catalogue numbers from Human Biolegend TotalSeq are A0251-A0257. Please find sequences for all of them below:
A0251 = GTCAACTCTTTAGCG <br>
A0252 = TGATGGCCTATTGGG <br>
A0253 = TTCCGCCTCTCTTTG <br>
A0254 = AGTAAGTTCAGCGTA <br>
A0255 = AAGTATCGTTTCGCA <br>
A0256 = GGTTGCCAGATGTCA <br>
A0257 = TGTCTTTCCTGCCAG

2. Islet 67: islet 67 GEX samples: SRR27326986, SRR27326987
A0251 = 24Hr Untreated <br>
A0252 = 24Hr Untreated <br>
A0253 = 24Hr Untreated <br>
A0254 = 24Hr Untreated <br>
A0255 = 2Hr H2O2 <br>
A0256 = 4Hr Cytokines <br>
A0257 = 24Hr Cytokines <br>
 
3. Islet 116 & Islet 117:
A0251 = 24Hr Untreated <br>
A0252 = 2Hr Cytokines <br>
A0253 = 4Hr Cytokines <br>
A0254 = 16h Cytokines <br>
A0255 = 24Hr Cytokines <br>
A0256 = 2Hr H2O2
 
4. Islet 150, Islet 162 & Islet 168:
* islet 150 GEX samples: SRR27326996, SRR27326997
* islet 162 GEX samples: SRR27326994, SRR27326995
* islet 168 GEX samples: SRR27326992, SRR27326993
A0251 = 2Hr Thapsigargin <br>
A0252 = 4Hr Thapsigargin <br>
A0253 = 4Hr DMSO <br>
A0254 = 24Hr Untreated <br>
A0255 = 24h DMSO <br>
A0256 = 24h Thapsigargin

## Obtain HTO data

HTO count matrices were downloaded from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE251912 and saved in a directory called `"/nfs/turbo/umms-scjp-pank/2_IIDP/results/gencode_v39_private/GSE251912/data/GEO/"` which is assumed as the variable `hto_dir` in the script `scripts/HTODemux.R`. If one uses this script, this path needs to be changed.

## Obtain gene expression profiles

We *do not* download the exprssion profiles from GEO but analyzed them from raw data instead, following the instructions in `1_run_processing_pipeline.ipynb` and `2_barcode_qc.ipynb`. From there, we can create a Seurat Rds file per sample using the following steps:

### Step 1: Get count matrix with only barcodes that satisfy QC criteria

```
    cat keep-barcodes.txt barcodes.tsv | sort | uniq -d > keep-in-cellbender.txt
    singularity exec /scratch/scjp_root/scjp99/vthihong/singularity-cache/porchard-mm-20230104.img mm subset --matrix matrix.mtx --features features.tsv --barcodes barcodes.tsv --keep-barcodes keep-in-cellbender.txt --prefix ${library}.
```

### Step 2: Get Seurat Rds file

```
library(Seurat)
library(ggplot2)
library(dplyr)
library(glue)
library(optparse)

RNA_MTX <- opts$matrix
RNA_FEATURES <- opts$features
RNA_BARCODES <- opts$barcodes
RESOLUTION <- opts$resolution
PCS <- opts$pcs
PREFIX <- opts$prefix
MARKERS <- strsplit(opts$markers, ',')[[1]]
SCTRANSFORM <- opts$sctransform
GET_MARKERS <- !opts$nomarkers

mm <- load_mm(RNA_MTX, RNA_FEATURES, RNA_BARCODES)

rna <- CreateSeuratObject(counts = mm, min.cells=5, min.features=5, assay = "RNA", project='RNA')

rna <- NormalizeData(rna, verbose=F)
rna <- FindVariableFeatures(rna, selection.method='vst', nfeatures=2000, verbose=F)
rna <- ScaleData(rna, verbose=F)

rna <- RunPCA(rna, npcs=200, verbose=F)

rna <- RunUMAP(rna, reduction='pca', dims=1:PCS)
rna <- FindNeighbors(rna, dims = 1:PCS, k.param = 20)
rna <- FindClusters(rna, resolution = RESOLUTION, n.start = 100)

saveRDS(rna, glue("{PREFIX}rna.rds"))
```

These Rds files are saved in a directory called `"/nfs/turbo/umms-scjp-pank/2_IIDP/results/gencode_v39_private/GSE251912/results/temp_rds/"` which is assumed as the variable `gex_dir` in the script `scripts/HTODemux.R`. If one uses this script, this path needs to be changed.

### Step 3: Merge data and associate barcodes to treatments

In [1]:
library(ggplot2)
library(dplyr)
library(Matrix)
library(Seurat)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching SeuratObject



In [2]:
files <- list.files("/nfs/turbo/umms-scjp-pank/2_IIDP/results/gencode_v39_private/GSE251912/results/singlets/", "_singlet.Rds")
files

[1] "Islet150_SRR27326996_singlet.Rds" "Islet150_SRR27326997_singlet.Rds"
[3] "Islet162_SRR27326994_singlet.Rds" "Islet162_SRR27326995_singlet.Rds"
[5] "Islet168_SRR27326992_singlet.Rds" "Islet168_SRR27326993_singlet.Rds"
[7] "Islet67_SRR27326986_singlet.Rds"  "Islet67_SRR27326987_singlet.Rds"

In [3]:
samples <- unlist(lapply(strsplit(files, "_"), '[', 2))
samples

[1] "SRR27326996" "SRR27326997" "SRR27326994" "SRR27326995" "SRR27326992"
[6] "SRR27326993" "SRR27326986" "SRR27326987"

In [6]:
for (i in files) {
    srr <- unlist(lapply(strsplit(i, "_"), '[', 2))
    rna <- readRDS(paste0("/nfs/turbo/umms-scjp-pank/2_IIDP/results/gencode_v39_private/GSE251912/results/singlets/", i))
    write.table(Cells(rna), paste0("/nfs/turbo/umms-scjp-pank/2_IIDP/results/gencode_v39/emptyDrops/results/pctMTusingBelowEndCliff_pctMtless30_FDR0.005/cellbender_default/", srr, "_passQC_barcodes_demultiplexed.csv"),
               row.names = F, col.names = F, quote = F)
}

In [5]:
data <- list()
for (i in files[1]) {
    srr <- unlist(lapply(strsplit(i, "_"), '[', 2))
    rna <- readRDS(paste0("/nfs/turbo/umms-scjp-pank/2_IIDP/results/gencode_v39_private/GSE251912/results/singlets/", i))
    rna@meta.data$SRR <- srr
    data[[srr]] <- rna
}
#merge Seurat objects for individual samples
merged_data <- merge(data[[samples[1]]], y=data[samples[2:length(samples)]], project='HTO', add.cell.ids=samples) 

In [12]:
#saveRDS(merged_data, "/nfs/turbo/umms-scjp-pank/2_IIDP/results/gencode_v39_private/GSE251912/results/singlets/merged_HTO.Rds")

In [4]:
merged_data <- readRDS("/nfs/turbo/umms-scjp-pank/2_IIDP/results/gencode_v39_private/GSE251912/results/singlets/merged_HTO.Rds")

In [5]:
hto_bc <- data.frame(id = c(paste0("A025", seq(1, 7))),
                    HTO_seq = c("GTCAACTCTTTAGCG", "TGATGGCCTATTGGG", "TTCCGCCTCTCTTTG", 
                                "AGTAAGTTCAGCGTA", "AAGTATCGTTTCGCA", "GGTTGCCAGATGTCA", "TGTCTTTCCTGCCAG"))
hto_bc

id,HTO_seq
<chr>,<chr>
A0251,GTCAACTCTTTAGCG
A0252,TGATGGCCTATTGGG
A0253,TTCCGCCTCTCTTTG
A0254,AGTAAGTTCAGCGTA
A0255,AAGTATCGTTTCGCA
A0256,GGTTGCCAGATGTCA
A0257,TGTCTTTCCTGCCAG


In [6]:
df <- merged_data@meta.data[, c("HTO_classification", "SRR")]
df$barcode <- unlist(lapply(strsplit(rownames(df), '_', fixed = TRUE), '[', 2))
df$HTO_seq <- sub(".*\\-", "", df$HTO_classification)

tail(df)

,HTO_classification,SRR,barcode,HTO_seq
,<chr>,<chr>,<chr>,<chr>
SRR27326987_CATACTTCACTCCCTA,HHTO5-AAGTATCGTTTCGCA,SRR27326987,CATACTTCACTCCCTA,AAGTATCGTTTCGCA
SRR27326987_GGGACAATCAACGAGG,HHTO3-TTCCGCCTCTCTTTG,SRR27326987,GGGACAATCAACGAGG,TTCCGCCTCTCTTTG
SRR27326987_GGTTCTCTCCTTCAGC,HHTO5-AAGTATCGTTTCGCA,SRR27326987,GGTTCTCTCCTTCAGC,AAGTATCGTTTCGCA
SRR27326987_GAGGGATCACAAATGA,HHTO7-TGTCTTTCCTGCCAG,SRR27326987,GAGGGATCACAAATGA,TGTCTTTCCTGCCAG
SRR27326987_TACCTGCAGGGAGGCA,HHTO2-TGATGGCCTATTGGG,SRR27326987,TACCTGCAGGGAGGCA,TGATGGCCTATTGGG
SRR27326987_CTATCCGCACCACTGG,HHTO1-GTCAACTCTTTAGCG,SRR27326987,CTATCCGCACCACTGG,GTCAACTCTTTAGCG


In [7]:
#Islet 67:
# islet 67 GEX samples: SRR27326986, SRR27326987
#A0251 = 24Hr Untreated
#A0252 = 24Hr Untreated
#A0253 = 24Hr Untreated
#A0254 = 24Hr Untreated
#A0255 = 2Hr H2O2
#A0256 = 4Hr Cytokines
#A0257 = 24Hr Cytokines

SRR27326986 <- data.frame(SRR = rep("SRR27326986", 7),
                        id = paste0("A025", seq(1, 7)),
                        Treatment = c(rep("24Hr_Untreated", 4),
                                     "2Hr_H2O2", "4Hr_Cytokines", "24Hr_Cytokines"))
SRR27326987 <- data.frame(SRR = rep("SRR27326987", 7),
                        id = paste0("A025", seq(1, 7)),
                        Treatment = c(rep("24Hr_Untreated", 4),
                                     "2Hr_H2O2", "4Hr_Cytokines", "24Hr_Cytokines"))
tmp <- rbind(SRR27326986, SRR27326987)

In [12]:
#Islet 150, Islet 162 & Islet 168:
# islet 150 GEX samples: SRR27326996, SRR27326997
# islet 162 GEX samples: SRR27326994, SRR27326995
# islet 168 GEX samples: SRR27326992, SRR27326993
#A0251 = 2Hr Thapsigargin
#A0252 = 4Hr Thapsigargin
#A0253 = 4Hr DMSO
#A0254 = 24Hr Untreated
#A0255 = 24h DMSO
#A0256 = 24h Thapsigargin

tmp2 <- data.frame(SRR = rep(c("SRR27326996", "SRR27326997", "SRR27326994", "SRR27326995", "SRR27326992", "SRR27326993"), each = 6),
                   id = rep(paste0("A025", seq(1, 6)), 6),
                   Treatment = rep(c("2Hr_Thapsigargin", "4Hr_Thapsigargin", "4Hr_DMSO", 
                                     "24Hr_Untreated", "24h_DMSO", "24h_Thapsigargin"), 6))
head(tmp2)

,SRR,id,Treatment
,<chr>,<chr>,<chr>
1,SRR27326996,A0251,2Hr_Thapsigargin
2,SRR27326996,A0252,4Hr_Thapsigargin
3,SRR27326996,A0253,4Hr_DMSO
4,SRR27326996,A0254,24Hr_Untreated
5,SRR27326996,A0255,24h_DMSO
6,SRR27326996,A0256,24h_Thapsigargin


In [13]:
map <- rbind(tmp, tmp2)
map <- inner_join(map, hto_bc)
head(map)

Joining with `by = join_by(id)`


,SRR,id,Treatment,HTO_seq
,<chr>,<chr>,<chr>,<chr>
1,SRR27326986,A0251,24Hr_Untreated,GTCAACTCTTTAGCG
2,SRR27326986,A0252,24Hr_Untreated,TGATGGCCTATTGGG
3,SRR27326986,A0253,24Hr_Untreated,TTCCGCCTCTCTTTG
4,SRR27326986,A0254,24Hr_Untreated,AGTAAGTTCAGCGTA
5,SRR27326986,A0255,2Hr_H2O2,AAGTATCGTTTCGCA
6,SRR27326986,A0256,4Hr_Cytokines,GGTTGCCAGATGTCA


In [10]:
dim(df)
df <- inner_join(df, map[, c("SRR", "Treatment", "HTO_seq")])
dim(df)

[1] 41294     4

Joining with `by = join_by(SRR, HTO_seq)`


[1] 41294     5

In [11]:
head(df)

,HTO_classification,SRR,barcode,HTO_seq,Treatment
,<chr>,<chr>,<chr>,<chr>,<chr>
1,0253-HHTO-03-TTCCGCCTCTCTTTG,SRR27326996,TGATCTTTCGCGTGAC,TTCCGCCTCTCTTTG,4Hr_DMSO
2,0252-HHTO-02-TGATGGCCTATTGGG,SRR27326996,TCATTTGGTTGCTCGG,TGATGGCCTATTGGG,4Hr_Thapsigargin
3,0253-HHTO-03-TTCCGCCTCTCTTTG,SRR27326996,GCAGGCTCACCAATTG,TTCCGCCTCTCTTTG,4Hr_DMSO
4,0254-HHTO-04-AGTAAGTTCAGCGTA,SRR27326996,TCGGGACTCTTTGGAG,AGTAAGTTCAGCGTA,24Hr_Untreated
5,0252-HHTO-02-TGATGGCCTATTGGG,SRR27326996,TCTCACGCATTGACTG,TGATGGCCTATTGGG,4Hr_Thapsigargin
6,0254-HHTO-04-AGTAAGTTCAGCGTA,SRR27326996,CTCCGATTCGAACCTA,AGTAAGTTCAGCGTA,24Hr_Untreated


In [ ]:
write.table(df, "/nfs/turbo/umms-scjp-pank/2_IIDP/results/gencode_v39_private/GSE251912/results/singlets/HTO_barcode_maps.txt",
           sep = "\t", quote = F, row.names = F)